In [34]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import ipywidgets as widgets

In [35]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
delta_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_time.csv', parse_dates=['Last_Update'])

In [36]:
# recovered_df.head()

In [37]:
# death_df.head()

In [38]:
# country_df.head()

In [39]:
#data cleaning

country_df.reset_index()
delta_df = delta_df[['Country_Region', 'Delta_Confirmed', 'Last_Update']]

# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
delta_df.columns = map(str.lower, delta_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = recovered_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
delta_df = delta_df.rename(columns={'last_update': 'date', 'country_region': 'country_name'})
# country_df.head()

In [40]:

# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [41]:
display(HTML("""<head>
  <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/css/bootstrap.min.css">
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.4.1/jquery.min.js"></script>
  <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.4.1/js/bootstrap.min.js"></script>
  <title>COVID-19 Dashboard | Hetav Desai</title>
</head>"""))
display(HTML(
"""
<div class="jumbotron text-center" style='background-color: #fff'>
  <h1 style="margin: auto; width: 100%;">COVID-19 Interactive Dashboard</h1>
  <h2></h2><p style="margin: auto; font-weight: bold; text-align: center; width: 100%;">Data Source: CSSE, John Hopkins University</p>
  <h2></h2><p style="margin: auto; font-weight: 400; text-align: center; width: 100%;">Last Updated: """ + str(country_df['last_update'][0]) + """</p>
  <h2></h2><p style="margin: auto; font-weight: 400; text-align: center; width: 100%;">( Best viewed on Desktop. You may need to scroll graphs horizontally on Mobile. )</p>
  <h2></h2><p style="margin: auto; font-weight: 400; text-align: center; width: 100%; color: #e73631;">Please wait for few seconds while the tables and graphs are loaded.<br><br>Please REFRESH the page if the graphs are not loaded in a few seconds.</p>
</div>

<div class="jumbotron text-center" style='padding: 0px'>
  <div class="row" style="background-color: #fff;width: 100%; margin: auto;">
    <div class="col-sm-4">
      <p style='text-align: center; background-color: #fff; font-weight: 400 ;color: #000'>Total Confirmed</p>
      <p style='text-align: center; font-size: 35px; font-weight: bold; color: #000'>""" + str(confirmed_total) + """</p>
    </div>
    <div class="col-sm-4">
      <p style='text-align: center; background-color: #fff; font-weight: 400 ;color: #000'>Total Deaths</p>
      <p style='text-align: center; font-size: 35px; font-weight: bold; color: #e73631'>""" + str(deaths_total) + """</p>
    </div>
    <div class="col-sm-4">
      <p style='text-align: center; background-color: #fff; font-weight: 400 ;color: #000'>Total Recovered</p>
      <p style='text-align: center; font-size: 35px; font-weight: bold; color: #70a82c'>""" + str(recovered_total) + """</p>
    </div>
  </div>
</div>
"""
))

# COVID-19 Confirmed, Death and Recovered Cases for Top 'n' number of countries with maximum cases

## Enter number of countries you want the data for

In [42]:
country_stats_df = country_df[['country', 'last_update','confirmed', 'deaths', 'recovered']]

In [43]:
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    red = 'color: #e73631'
    grey = 'color: #000'
    green = 'color: #70a82c'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 2] = grey
    df1.iloc[:, 3] = red
    df1.iloc[:, 4] = green
    
    return df1

def show_latest_cases(n):
    if n=='':
        print("Enter valid number")
    else:
        n = int(n)
        if n>0:
            return country_stats_df.sort_values('confirmed', ascending= False).reset_index(drop=True).head(n).style.apply(highlight_col, axis=None).set_properties(**{'text-align': 'left'}).set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
numbers_list = []
for i in range(1,11):
    numbers_list.append(str(i))
dd_num = widgets.Dropdown(
    options = numbers_list,
    value = '10',
    description = 'Select No.: '
)

interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

In [44]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

# Bubble Chart for 'n' worst hit countries
### Slide to select value of n

In [45]:
# # plotting the 20 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700,
    )
    fig.show();

interact(bubble_chart, n=10);

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

display(HTML(
"""
<div class="jumbotron text-center" style='background-color: #fff'>
  <h2></h2><p style="margin: auto; font-weight: 400; text-align: center; width: 100%; color: #e73631;">DRAG over graph to ZOOM IN selected region. DOUBLE TAP to ZOOM OUT.</p>
</div>
"""
))

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

In [46]:
def plot_cases_of_a_country(Country):
    country = Country
    labels = ['confirmed', 'deaths', 'recovered']
    colors = ['black', 'red', 'green']
    mode_size = [6, 8, 8]
    line_size = [4, 5, 5]
    
    df_list = [confirmed_df, death_df, recovered_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        width = 800,
        paper_bgcolor='#f5f5f5',
        plot_bgcolor='rgba(0,0,0,0)'
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

In [47]:
# delta_df.head()

In [48]:
delta_pivoted_df = delta_df.pivot(index='date', columns='country_name', values='delta_confirmed')
delta_pivoted_df.reset_index(level=0, inplace=True)
delta_world_df = pd.DataFrame()
delta_world_df['World'] = delta_pivoted_df[delta_pivoted_df.columns].sum(axis=1)
delta_world_df['date'] = delta_pivoted_df['date']

In [49]:
# delta_world_df.head()

In [50]:
def plot_new_cases_of_country(Country):
    country = Country
    if(country == 'World' or country == 'world'):
        y_data = np.array(list(delta_world_df[country]))
    else:
        y_data = np.array(list(delta_pivoted_df[country]))
    x_data = np.array(list(delta_df['date']))
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=x_data,
        y=y_data,
        name='Daily Increase',
        marker_color='crimson',
        hovertemplate='Date: %{x}; \n  New Cases: %{y}',
    ))
    fig.update_layout(
            title="Daily increase in cases of " + country,
            xaxis_title='Date',
            yaxis_title='No. of New Cases',
            margin=dict(l=20, r=20, t=40, b=20),
            width = 800,
            paper_bgcolor='#f5f5f5',
            plot_bgcolor='rgba(0,0,0,0)',
            dragmode='none',
    );
    fig.update_yaxes(type="linear")
    fig.show()    

# Overview of Confirmed Cases and Deaths for Specific Country

### Enter Name of any Country with capital initial (e.g. India, US, Italy, China, United Kingdom)
### Or Enter "World" for Global Stats

In [51]:
# df_cols = list(country_df.country.values)
# country_list = ['World']
# country_list = country_list + (df_cols)
# dd_country = widgets.Dropdown(
#         options=country_list,
#         value=country_list[0],
#         description='X'
#     )

# country_input = widgets.Dropdown(
#         options=country_list,
#         description='Select Country'
#     )
# interact(plot_cases_of_a_country, Country='World')

# ipywLayout = widgets.Layout(border='solid 2px green')
# ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
# widgets.VBox([fig], layout=ipywLayout)

In [52]:
interact(plot_cases_of_a_country, Country='World');
display(HTML(
"""
<div class="jumbotron text-center" style='background-color: #fff'>
  <h2></h2><p style="margin: auto; font-weight: 400; text-align: center; width: 100%; color: #e73631;">DRAG over graph to ZOOM IN selected region. DOUBLE TAP to ZOOM OUT.</p>
</div>
"""
))

interactive(children=(Text(value='World', description='Country'), Output()), _dom_classes=('widget-interact',)…

In [53]:
# Daily increase

In [54]:
# interact(plot_new_cases_of_country, Country='India')

# COVID-19 in different parts of the world

In [55]:
world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map